In [2]:
import numpy as np
import pandas as pd
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP

from sklearn.linear_model import LogisticRegression
from glob import glob

from scipy.signal import butter, lfilter, convolve, boxcar

In [3]:
def creat_mne_raw_object(fname,read_events=True):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = read_montage('standard_1005',ch_names)

    ch_type = ['eeg']*len(ch_names)
    data = 1e-6*np.array(data[ch_names]).T
    
    if read_events:
        # events file
        ev_fname = fname.replace('_data','_events')
        # read event file
        events = pd.read_csv(ev_fname)
        events_names = events.columns[1:]
        events_data = np.array(events[events_names]).T
        
        # define channel type, the first is EEG, the last 6 are stimulations
        ch_type.extend(['stim']*6)
        ch_names.extend(events_names)
        # concatenate event file and data
        data = np.concatenate((data,events_data))
        
    # create and populate MNE info structure
    info = create_info(ch_names,sfreq=500.0, ch_types=ch_type, montage=montage)
    info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    
    return raw

In [4]:
subjects = range(1,13)
ids_tot = []
pred_tot = []

# design a butterworth bandpass filter 
freqs = [7, 30]
b,a = butter(5,np.array(freqs)/250.0,btype='bandpass')

# CSP parameters
# Number of spatial filter to use
nfilters = 4

# convolution
# window for smoothing features
nwin = 250

# training subsample
subsample = 10

# submission file
submission_file = 'beat_the_benchmark.csv'
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [5]:
subject = subjects[0]

In [6]:
epochs_tot = []
y = []

################ READ DATA ################################################
fnames =  glob('train/subj%d_series*_data.csv' % (subject))
fnames

# read and concatenate all the files
raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])

# pick eeg signal
picks = pick_types(raw.info,eeg=True)

# Filter data for alpha frequency and beta band
# Note that MNE implement a zero phase (filtfilt) filtering not compatible
# with the rule of future data.
# Here we use left filter compatible with this constraint
raw._data[picks] = lfilter(b,a,raw._data[picks])

################ CSP Filters training #####################################
# get event posision corresponding to Replace
events = find_events(raw,stim_channel='Replace', verbose=False)
# epochs signal for 1.5 second before the movement
epochs = Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False,
                picks=picks, baseline=None, preload=True,
                add_eeg_ref=False, verbose=False)

epochs_tot.append(epochs)
y.extend([1]*len(epochs))

# epochs signal for 1.5 second after the movement, this correspond to the 
# rest period.
epochs_rest = Epochs(raw, events, {'after' : 1}, 0.5, 2, proj=False,
                picks=picks, baseline=None, preload=True,
                add_eeg_ref=False, verbose=False)

# Workaround to be able to concatenate epochs with MNE
epochs_rest.times = epochs.times

y.extend([-1]*len(epochs_rest))
epochs_tot.append(epochs_rest)

# Concatenate all epochs
epochs = concatenate_epochs(epochs_tot)

# get data 
X = epochs.get_data()
y = np.array(y)

In [7]:
print X.shape

(519, 32, 751)


In [8]:
# train CSP
csp = CSP(n_components=nfilters, reg='lws')
csp.fit(X,y)

In [9]:
csp.filters_

array([[ 0.14284158, -0.0124476 , -1.15181086, ..., -0.32848577,
         0.27633894,  0.21076838],
       [ 0.03545028, -0.17627069,  0.38238997, ...,  1.23755569,
        -1.10641096, -0.43786365],
       [-0.15390217,  1.27869443,  1.38828934, ..., -1.81040206,
         2.49710614, -1.87747311],
       ..., 
       [-0.53817737,  1.18162952,  1.33437625, ..., -2.51403004,
         5.27909561, -1.26719652],
       [ 0.96766153, -1.34270016, -0.79163653, ..., -1.69135517,
         1.61219612, -0.09232703],
       [-0.88471982,  2.13878891, -1.31029558, ..., -4.43521648,
        -1.39382222,  2.17728606]])

In [10]:
csp.filters_[0:nfilters]

array([[  1.42841584e-01,  -1.24476011e-02,  -1.15181086e+00,
         -4.81122274e-01,   2.17852531e+00,  -1.78174471e-01,
         -1.85794243e-01,  -2.47393150e+00,  -1.06285101e+01,
          4.05632791e-01,  -1.95090674e-01,   1.36002630e+00,
          2.93879680e+01,  -8.57205448e+00,   1.14897067e+00,
         -3.49279721e-02,  -1.19594992e+00,  -5.65571685e+00,
         -9.30714813e-01,  -9.08427002e-01,   2.96800092e-01,
         -1.14316385e-01,   8.49736442e-02,  -2.57584082e+00,
         -2.34278313e+00,   1.88698035e+00,  -1.01862718e+00,
         -6.48087281e-02,   6.28525769e-01,  -3.28485770e-01,
          2.76338945e-01,   2.10768383e-01],
       [  3.54502826e-02,  -1.76270688e-01,   3.82389969e-01,
         -2.09119615e-01,  -2.33003157e+00,  -6.86454664e-02,
          1.22396198e-01,  -7.41017964e-01,   2.12235085e+00,
          1.10729580e+00,   8.46586088e-01,   5.88684507e-01,
          7.63932238e+00,   8.82907558e+00,  -6.94349270e+00,
         -5.87981201e-02,

In [11]:
np.empty((32,32))

array([[ -2.68156159e+154,  -2.68156159e+154,   1.36559745e-320, ...,
          3.70079764e-033,   1.39642638e-076,   1.10711049e-047],
       [  1.21111753e-099,   6.01391519e-154,   1.28446940e-057, ...,
          6.00458986e-067,   6.01353868e-154,   1.39805386e-076],
       [  1.02455700e+136,   6.01347002e-154,   4.90843030e-062, ...,
          9.15604172e-072,   6.01347176e-154,   3.84568673e-086],
       ..., 
       [  6.51583213e-014,   3.75586017e-016,   2.04062841e-017, ...,
          1.73517739e-018,   3.35060057e-016,   2.63541432e-016],
       [  3.05218684e-016,   6.07453520e-016,   5.90540564e-016, ...,
          1.69575181e-015,   3.50878105e-016,   9.82859278e-017],
       [  2.70186284e-015,   1.09798338e-016,   1.37155367e-016, ...,
          1.00534612e-015,   1.18070108e-016,   2.52930604e-016]])

In [12]:
nwin

250

In [14]:
x_test = np.array(range(10)) 
print x_test
conv_test = boxcar(3)
print conv_test
test_result = convolve(x_test, conv_test, 'full')
print test_result

[0 1 2 3 4 5 6 7 8 9]
[ 1.  1.  1.]
[  0.   1.   3.   6.   9.  12.  15.  18.  21.  24.  17.   9.]


In [15]:
################ Create Training Features #################################x`
# apply csp filters and rectify signal
feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2

In [16]:
feat.shape

(4, 1422392)

In [18]:
conv_result = convolve(feat[0],boxcar(nwin),'full')

In [19]:
np.log(conv_result).shape

(1422641,)

In [20]:
# smoothing by convolution with a rectangle window    
feattr = np.empty(feat.shape)
for i in range(nfilters):
    feattr[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]

In [21]:
feattr[0].shape

(1422392,)

In [22]:
# training labels
# they are stored in the 6 last channels of the MNE raw object
labels = raw._data[32:]

In [23]:
labels.shape

(6, 1422392)

In [28]:
################ Create test Features #####################################
# read test data 
fnames =  glob('test/subj%d_series*_data.csv' % (subject))
raw = concatenate_raws([creat_mne_raw_object(fname, read_events=False) for fname in fnames])
raw._data[picks] = lfilter(b,a,raw._data[picks])

In [29]:
raw._data.shape

(32, 233081)

In [30]:
# read ids
ids = np.concatenate([np.array(pd.read_csv(fname)['id']) for fname in fnames])
ids_tot.append(ids)

In [34]:
ids

array(['subj1_series10_0', 'subj1_series10_1', 'subj1_series10_2', ...,
       'subj1_series9_115950', 'subj1_series9_115951',
       'subj1_series9_115952'], dtype=object)

In [36]:
# apply preprocessing on test data
feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2
featte = np.empty(feat.shape)
for i in range(nfilters):
    featte[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]

In [37]:
cols

['HandStart',
 'FirstDigitTouch',
 'BothStartLoadPhase',
 'LiftOff',
 'Replace',
 'BothReleased']

In [46]:
feattr.shape

(4, 1422392)

In [47]:
feattr[:,::subsample].shape

(4, 142240)

In [55]:
n = np.vstack((np.array(range(10)), np.array(range(10))))

In [70]:
n[:, ::3]

array([[0, 3, 6, 9],
       [0, 3, 6, 9]])

In [71]:
subsample

10

In [73]:
labels.shape

(6, 1422392)

In [80]:
lr = LogisticRegression()
pred = np.empty((len(ids),6))

# Predict HandStart
print('Train subject %d, class %s' % (subject, cols[i]))
lr.fit(feattr[:,::subsample].T,labels[0,::subsample])
result = lr.predict_proba(featte.T)

Train subject 1, class LiftOff


In [91]:
lr.classes_

array([ 0.,  1.])

In [92]:
pred_tot

[]

In [93]:
################ Train classifiers ########################################
lr = LogisticRegression()
pred = np.empty((len(ids),6))
for i in range(6):
    print('Train subject %d, class %s' % (subject, cols[i]))
    lr.fit(feattr[:,::subsample].T,labels[i,::subsample])
    pred[:,i] = lr.predict_proba(featte.T)[:,1]

pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased


In [94]:
pred_tot

[array([[  9.99958646e-01,   9.99366322e-01,   9.99303819e-01,
           9.99959764e-01,   6.90948026e-13,   1.73640689e-11],
        [  9.98650507e-01,   9.90668262e-01,   9.89917561e-01,
           9.98941404e-01,   2.63710147e-10,   3.26015263e-09],
        [  9.84580971e-01,   9.41299682e-01,   9.37476842e-01,
           9.89653133e-01,   1.66851374e-08,   1.26267427e-07],
        ..., 
        [  1.48022111e-02,   1.69821707e-02,   1.67152722e-02,
           1.49918511e-02,   4.63412703e-02,   3.97751981e-02],
        [  1.49211494e-02,   1.70638440e-02,   1.67997124e-02,
           1.50567082e-02,   4.59911218e-02,   3.94364507e-02],
        [  1.50668325e-02,   1.71588123e-02,   1.68964938e-02,
           1.51281984e-02,   4.55630749e-02,   3.90238460e-02]])]

In [95]:
ids_tot

[array(['subj1_series10_0', 'subj1_series10_1', 'subj1_series10_2', ...,
        'subj1_series9_115950', 'subj1_series9_115951',
        'subj1_series9_115952'], dtype=object)]

In [97]:
# create pandas object for sbmission
submission = pd.DataFrame(index=np.concatenate(ids_tot), columns=cols, data=np.concatenate(pred_tot))

In [98]:
submission

,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
subj1_series10_0,0.999959,0.999366,0.999304,0.999960,6.909480e-13,1.736407e-11
subj1_series10_1,0.998651,0.990668,0.989918,0.998941,2.637101e-10,3.260153e-09
subj1_series10_2,0.984581,0.941300,0.937477,0.989653,1.668514e-08,1.262674e-07
subj1_series10_3,0.907491,0.792142,0.782242,0.943971,3.830773e-07,2.012880e-06
subj1_series10_4,0.688652,0.550710,0.537572,0.811159,4.465582e-06,1.773868e-05
subj1_series10_5,0.397326,0.329375,0.318836,0.588873,3.187758e-05,1.017563e-04
subj1_series10_6,0.195965,0.188962,0.182297,0.369743,1.565362e-04,4.202677e-04
subj1_series10_7,0.096735,0.112270,0.108282,0.220592,5.710918e-04,1.335967e-03
subj1_series10_8,0.051468,0.071219,0.068766,0.135068,1.633088e-03,3.424545e-03
subj1_series10_9,0.030078,0.048552,0.046950,0.087977,3.803350e-03,7.319690e-03


In [99]:
# write file
submission.to_csv(submission_file,index_label='id',float_format='%.3f')